# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## **🗒️ In this notebook we will see how to create a training dataset from the feature groups:** 
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![tutorial-flow](../images/02_training-dataset.png) 

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/146
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🔪 Feature Selection </span>

We start by selecting all the features we want to include for model training/inference.

In [4]:
# Load feature groups.
trans_fg = fs.get_feature_group('transactions', version=1)

# Select features for training data.
ds_query = trans_fg.select(["fraud_label", "loc_delta_t_plus_1", "loc_delta_t_minus_1", "time_delta_t_plus_1", "time_delta_t_minus_1", "country"])

ds_query.show(5)

2022-06-18 14:52:38,291 INFO: USE `davit000_featurestore`
2022-06-18 14:52:39,707 INFO: SELECT `fg0`.`fraud_label` `fraud_label`, `fg0`.`loc_delta_t_plus_1` `loc_delta_t_plus_1`, `fg0`.`loc_delta_t_minus_1` `loc_delta_t_minus_1`, `fg0`.`time_delta_t_plus_1` `time_delta_t_plus_1`, `fg0`.`time_delta_t_minus_1` `time_delta_t_minus_1`, `fg0`.`country` `country`
FROM `davit000_featurestore`.`transactions_1` `fg0`


,fraud_label,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_plus_1,time_delta_t_minus_1,country
0,0,0.639831,0.102947,0.057257,0.050579,US
1,0,0.091154,0.099326,0.193160,0.019734,US
2,0,0.158561,0.294690,0.007222,0.140729,US
3,0,0.170991,0.318806,0.069201,0.017454,US
4,0,0.440275,0.641636,0.445521,0.161840,US


Recall that we computed the features in `transactions_4h_aggs` using 4-hour aggregates. If we had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join we would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

v🤖 Transformation Functions </span>

We will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [5]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = {
    "loc_delta_t_plus_1": min_max_scaler, 
    "loc_delta_t_minus_1": min_max_scaler, 
    "time_delta_t_plus_1": min_max_scaler, 
    "time_delta_t_minus_1": min_max_scaler,
    "country": label_encoder
}

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.
In order to create a Feature View we may use `fs.create_feature_view()`

In [6]:
feature_view = fs.create_feature_view(
    name='transactions_view',
    query=ds_query,
    labels=["fraud_label"],
    transformation_functions=transformation_functions
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/146/fs/88/fv/transactions_view/version/1


To view and explore data in the feature view we can retrieve batch data using `get_batch_data()` method 

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_training_dataset()` method.

**From feature view APIs we can also create training datasts based on even time filters specifing `start_time` and `end_time`** 



In [7]:
from datetime import datetime
date_format = "%Y-%m-%d %H:%M:%S"

In [8]:
# Create training datasets based event time filter
start_time = int(float(datetime.strptime("2022-01-01 00:00:01", date_format).timestamp()) * 1000)
end_time = int(float(datetime.strptime("2022-02-28 23:59:59", date_format).timestamp()) * 1000)

td_jan_feb_version, td_job = feature_view.create_training_data(
        start_time = start_time,
        end_time = end_time,    
        description = 'transactions_dataset_jan_feb',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
    )

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/146/jobs/named/transactions_view_1_1_create_fv_td_18062022125419/executions


In [9]:
start_time = int(float(datetime.strptime("2022-03-01 00:00:01", date_format).timestamp()) * 1000)
end_time = int(float(datetime.strptime("2022-03-31 23:59:59", date_format).timestamp()) * 1000)

td_mar_version, td_job = feature_view.create_training_data(
        start_time = start_time,
        end_time = end_time,
        description = 'transactions_dataset_mar',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
 )


Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/146/jobs/named/transactions_view_1_2_create_fv_td_18062022125524/executions


## <span style="color:#ff5f27;"> 🪝 Training Dataset retreival </span>

To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset_splits` or `get_training_dataset` methods. If version is not provided or provided version has not already existed, it creates a new version of training data according to given arguments and returns a dataframe. If version is provided and has already existed, it reads training data from storage or feature groups and returns a dataframe. If split is provided, it reads the specific split.

In [10]:
train_jan_feb_x, train_jan_feb_y = feature_view.get_training_data(td_jan_feb_version)

In [11]:
train_mar_x, train_mar_y = feature_view.get_training_data(td_mar_version)

In [12]:
train_jan_feb_x

,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_plus_1,time_delta_t_minus_1,country
0,0.220553,0.035485,0.006817,0.006020,0
1,0.031421,0.034237,0.022997,0.002348,0
2,0.054657,0.101580,0.000860,0.016753,0
3,0.058941,0.109893,0.008239,0.002077,0
4,0.151765,0.221174,0.053042,0.019267,0
...,...,...,...,...,...
272373,0.341645,0.076711,0.041343,0.020512,0
272374,0.105838,0.052382,0.068046,0.058096,0
272375,0.047069,0.161564,0.025338,0.024880,0
272376,0.183549,0.017002,0.005801,0.007575,0


In [13]:
train_mar_x

,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_plus_1,time_delta_t_minus_1,country
0,0.195343,0.064894,0.027908,0.111412,0
1,0.196592,0.207137,0.136705,0.011971,0
2,0.399929,0.016068,0.029227,0.104640,0
3,0.000045,0.190295,0.058226,0.008824,0
4,0.087117,0.054528,0.022742,0.000952,0
...,...,...,...,...,...
84798,0.045101,0.060464,0.122956,0.015121,0
84799,0.136333,0.079189,0.326743,0.185223,0
84800,0.078657,0.038336,0.055312,0.013967,0
84801,0.077060,0.078857,0.075310,0.004124,0


The feature view and training dataset are now visible in the UI

![fg-overview](../images/fv_overview.gif)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the following notebook, we will train a model on the dataset we created in this notebook and have quick overview of the lineage.